# Motion Detection


Ovaj zadatak je Python skripta za detekciju osoba u stvarnom vremenu koristeći OpenCV biblioeku koja se koristi za učitavanje mreža dubokog učenja definisanih u Caffe frameworku.

Caffe (Convolutional Architecture for Fast Feature Embedding) predstavlja značajan okvir za duboko učenje, razvijen na Univerzitetu Kalifornija, Berkeley. Od svog nastanka, Caffe je prepoznat po svojoj izuzetnoj brzini, modularnosti i jednostavnosti korištenja.

Jedna od najistaknutijih karakteristika Caffe-a je njegova brzina. Zahvaljujući optimizaciji za GPU, Caffe može procesirati više od nekoliko miliona slika dnevno na grafičkoj kartici. Ova performansa čini ga idealnim za aplikacije koje zahtjevaju obradu velikih količina podataka u realnom vremenu.

Caffe-ova modularna arhitektura omogućava lako eksperimentisanje sa različitim konfiguracijama neuronskih mreža. Definicije modela se pišu u čitljivom i jasnom protokolnom pufer formatu, što olakšava specifikaciju i modifikaciju mrežnih arhitektura. Pored toga, Caffe podržava širok spektar zadataka dubokog učenja, uključujući klasifikaciju slika, segmentaciju, detekciju objekata i obradu video zapisa.


# 1. Kreiranje klase kamera


Potrebno je kreirati klasu kamera i definisati instancu u main funkciji. Unutar klase definišu se osnovni paramteri za pokretanje i prikaz kamere.

In [1]:
import cv2 as cv

class Camera:
    cap = cv.VideoCapture(0)
    
    def __init__(self):
        pass  # Trenutno, nema logike inicijalizacije
    
    def run(self):
        print("Kamera je pokrenuta...")
        while True:
            ret, frame = self.cap.read()
            if not ret:
                break

            # Prikazivanje trenutnog kadra u prozoru
            cv.imshow('Camera Feed', frame)

            # Ako je pritisnuta tipka 'q', prekini petlju
            if cv.waitKey(1) & 0xFF == ord('q'):
                break

        # Oslobađanje resursa i zatvaranje svih OpenCV prozora
        cv.destroyAllWindows()

def main():
    camera = Camera()
    camera.run()

if __name__ == "__main__":
    main()


Kamera je pokrenuta...


# 2. Implementacija detekcije 

Implementacija detekcije osobe koristeći Caffe framework model je urađena kroz sljedeće korake:
1. Učitava pretreniran model koristeći readNetFromCaffe.
2. Pretprocesira trenutni kadar iz video snimka u blob format koji je pogodan za mrežu.
3. Prolazi kroz mrežu koristeći forward da bi dobio predikcije.
4. Obradjuje rezultate predikcije, provjerava pouzdanost detekcije i crta pravougaonik oko detektovane osobe.

### 2.1 Učitavanje pretreniranog modela


```python
net = cv.dnn.readNetFromCaffe('models/config.txt', 'models/mobilenet_iter_73000.caffemodel')
```

- cv.dnn.readNetFromCaffe(prototxt, caffemodel) - funkcija iz OpenCV biblioteke koja učitava model iz Caffe formata.
- prototxt fajl ('models/config.txt') - sadrži definiciju strukture neuronske mreže (slojevi, parametri slojeva).
- caffemodel fajl ('models/mobilenet_iter_73000.caffemodel') - sadrži unapred istrenirane težine mreže.

### 2.2 Pretprocesiranje kadra

```python
blob = cv.dnn.blobFromImage(frame, scalefactor , size , mean)
```

cv.dnn.blobFromImage konvertuje sliku (kadar iz video snimka) u format koji može biti korišten kao ulaz za neuronsku mrežu.

Parametri funkcije:
 1. frame: trenutni kadar iz video snimka.
 2. scalefactor: skala faktora za normalizaciju piksela.
 3. size: dimenzije na koje se slika skalira.
 4. mean: vrijednost koja se oduzima od svakog kanala boje (za centriranje podataka).
 

### 2.3 Postavljanje ulaza u mrežu i izvršavanje predikcije 

```python
self.net.setInput(blob)
```

- setInput postavlja pretprocesirani kadar (blob) kao ulaz u neuronsku mrežu.

```python
detections = self.net.forward()
```

- forward metoda izvršava prolaz unapred kroz mrežu, tj. vrši predikciju na osnovu ulaza (blob).

Rezultat je niz detekcija koji mreža vraća.

### 2.4 Obrada rezultata predikcije

```python

for i in range(detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    idx = int(detections[0, 0, i, 1])
    if idx == 15 and confidence > 0.5:
        box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
        (startX, startY, endX, endY) = box.astype("int")
        cv.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
        person_detected = True

```

- detections.shape[2] - daje broj detekcija.
- Svaka detekcija ima niz atributa:
    - detections[0, 0, i, 2] - pouzdanost detekcije (confidence).
    - detections[0, 0, i, 1] - indeks klase detektovanog objekta.
- Ako je detektovana osoba (indeks klase 15) i pouzdanost je veća od 0.5:
    - detections[0, 0, i, 3:7] - koordinate pravougaonika oko detektovanog objekta, skalirane na dimenzije trenutnog kadra.
    - cv.rectangle - crta pravougaonik oko detektovane osobe na kadru.


### 2.5 Oslobađanje resursa


```python
    self.cap.release()
        cv.destroyAllWindows()
        print("Camera released...")
```

Oslobađaju se resursi kamere i zatvaraju svi OpenCV prozori. 

### 2.6 Puna implementacija 

In [2]:
import cv2 as cv
import numpy as np


class Camera:
    
    net = cv.dnn.readNetFromCaffe('models/config.txt', 'models/mobilenet_iter_73000.caffemodel')
    cap = cv.VideoCapture(0)
    
    def __init__(self):
        pass  # Trenutno, nema logike inicijalizacije
    
    def run(self):
        print("Kamera je pokrenuta...")
        while True:
            ret, frame = self.cap.read()
            if not ret:
                break

            # Pretvori kadar u blob format za unos u neuralnu mrežu
            blob = cv.dnn.blobFromImage(frame, 0.007843, (300, 300), 127.5)
            
            # Postavi blob kao unos u mrežu
            self.net.setInput(blob)

            # Izvrši predikciju koristeći mrežu
            detections = self.net.forward()
            person_detected = False

            # Prođi kroz sve detekcije
            for i in range(detections.shape[2]):
                # Dobavi pouzdanost detekcije (konfidenciju)
                confidence = detections[0, 0, i, 2]
                
                # Dobavi indeks klase detektovanog objekta
                idx = int(detections[0, 0, i, 1])
                
                # Ako je detektovan objekat klase '15' (osoba) i konfidencija je veća od 0.5
                if idx == 15 and confidence > 0.5:
                    # Izračunaj koordinate pravougaonika oko detektovane osobe
                    box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
                    (startX, startY, endX, endY) = box.astype("int")
                    
                    # Nacrtaj pravougaonik oko detektovane osobe
                    cv.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
                    
                    # Označi da je osoba detektovana
                    person_detected = True

            # Prikazivanje trenutnog kadra u prozoru
            cv.imshow('Camera Feed', frame)

            # Ako je pritisnuta tipka 'q', prekini petlju
            if cv.waitKey(1) & 0xFF == ord('q'):
                break

        # Oslobađanje resursa i zatvaranje svih OpenCV prozora
        self.cap.release()
        cv.destroyAllWindows()
        print("Camera released...")

def main():
    camera = Camera()
    camera.run()

if __name__ == "__main__":
    main()

Kamera je pokrenuta...
Camera released...


# 3. Snimanje i spremanje videozapisa

###  3.1 Definisanje varijable za izlaz video zapisa i postavljanje međuspremnika za detekciju

```python

out = None

```

out je varijabla koja će držati objekat cv.VideoWriter kada je snimanje aktivno. Orginalno je postavljena na None što znači da snimanje nije aktivno

```python

buffer_duration = 5
self.buffer_size = int(self.frame_rate * self.buffer_duration)
self.buffer = deque(maxlen=self.buffer_size)

```
- buffer_duration - definiše koliko sekundi unazad se čuva informacija o detekcijama.
- buffer_size - izračunava koliko frejmova treba čuvati na osnovu brzine kadrova (frame rate) i trajanja međuspremnika.
- buffer je deque (dvosmjerna lista) sa maksimalnom dužinom buffer_size, koja čuva informacije o detekcijama osoba.


###  3.2 Dodavanje rezultata detekcije u međuspremnik

```python

self.buffer.append(person_detected)

```
Nakon što se izvrši predikcija i proveri da li je osoba detektovana (person_detected), rezultat (True ili False) se dodaje u međuspremnik buffer.


###  3.2 Provjera i pokretanje snimanja

 ```python

if sum(self.buffer) > 0:
    if self.out is None:
        now = datetime.datetime.now()
        formatted_now = now.strftime("%d-%m-%y-%H-%M-%S")
        print("Person motion detected at", formatted_now)
        current_recording_name = os.path.join('detected_videos', f'{formatted_now}.mp4')
        fourcc = cv.VideoWriter_fourcc(*'mp4v')
        self.out = cv.VideoWriter(current_recording_name, fourcc, self.frame_rate, (frame.shape[1], frame.shape[0]))
    self.out.write(frame)
else:
    if self.out is not None:
        self.out.release()
        self.out = None
        print("Recording stopped")

 
```

- Provjera detekcije u međuspremniku:

     -  if sum(self.buffer) > 0 - provjerava da li postoji bilo koja detekcija osobe u poslednjih buffer_duration sekundi.

- Pokretanje snimanja:

    - if self.out is None: provjerava da li je snimanje već aktivno.

    - Ako nije, postavlja se novi video zapis:
    
        - datetime.datetime.now() - dobija trenutni datum i vrijeme.
        - now.strftime("%d-%m-%y-%H-%M-%S") - formatira datum i vrijeme za naziv fajla.
        - current_recording_name = os.path.join('detected_videos', f'{formatted_now}.mp4') -  kreira naziv za snimljeni video fajl u direktorijumu detected_videos formata MP4
        - cv.VideoWriter_fourcc(*'mp4v') - definiše codec za snimanje (MP4 format).
        - self.out = cv.VideoWriter(current_recording_name, fourcc, self.frame_rate, (frame.shape[1], frame.shape[0])) - inicijalizuje objekat VideoWriter sa zadatim nazivom 'current_recording_name', brzinom kadrova '(self.frame_rate)' i veličinom kadra '(horizontalno, vertikalno)'.
        - self.out.write(frame) - zapisuje trenutni kadar u video fajl.

- Zaustavljanje snimanja:

    - if sum(self.buffer) == 0: - provjerava da li u poslednjih buffer_duration sekundi nije bilo detekcije osoba.
    - if self.out is not None: - provjerava da li je snimanje aktivno.
    - self.out.release() - oslobađa resurse video zapisivača i zaustavlja snimanje.
    - self.out = None - postavlja out na None, što znači da snimanje više nije aktivno.


### 3.3 Spremanje u direktorij

 ```python
if not os.path.exists('detected_videos'):
    os.makedirs('detected_videos')
```
Provjeri da li postoji direktorijum 'detected_videos' i ako ne postoji, kreira ga.

### 3.4 Oslobađanje resursa

Prethodno oslobađanje resura se zamjenjuje sa novim

 ```python
if self.out is not None:
    self.out.release()
    self.out = None

self.cap.release()
cv.destroyAllWindows()
print("Camera released...")

 ```

Oslobađa se video  zapisivanje i zatvaraju svi OpenCV prozori.

# 4. Puna implementacija

In [ ]:
import cv2 as cv
import numpy as np
import datetime
import os
from collections import deque

# Provjeri da li postoji direktorijum 'detected_videos', i ako ne postoji, kreiraj ga
if not os.path.exists('detected_videos'):
    os.makedirs('detected_videos')

class Camera:
    # Učitaj pretreniran model iz Caffe formata sa zadatim konfiguracionim fajlom i težinama modela
    net = cv.dnn.readNetFromCaffe('models/config.txt', 'models/mobilenet_iter_73000.caffemodel')
    cap = cv.VideoCapture(0)
    
    # Inicijalno nije postavljen izlaz za video zapis
    out = None
    
    # Definiši trajanje međuspremnika u sekundama
    buffer_duration = 5  
    


    def __init__(self):
        # Postavi brzinu kadrova za kameru
        self.frame_rate = (self.cap.get(cv.CAP_PROP_FPS)*(1/2))
        # Izračunaj veličinu međuspremnika na osnovu brzine kadrova i trajanja međuspremnika
        self.buffer_size = int(self.frame_rate * self.buffer_duration)
        # Inicijalizuj deque (dvosmjernu listu) sa maksimalnom dužinom da bi veličina međuspremnika bila fiksna
        self.buffer = deque(maxlen=self.buffer_size)


    def run(self):
        print("Camera started...")
        while True:
            # Čitaj trenutni kadar sa kamere
            ret, frame = self.cap.read()
            if not ret:
                break
            
            # Pretvori kadar u blob format za unos u neuralnu mrežu
            blob = cv.dnn.blobFromImage(frame, 0.007843, (300, 300), 127.5)
            
            # Postavi blob kao unos u mrežu
            self.net.setInput(blob)

            # Izvrši predikciju koristeći mrežu
            detections = self.net.forward()
            person_detected = False

            # Prođi kroz sve detekcije
            for i in range(detections.shape[2]):
                # Dobavi pouzdanost detekcije (konfidenciju)
                confidence = detections[0, 0, i, 2]
                
                # Dobavi indeks klase detektovanog objekta
                idx = int(detections[0, 0, i, 1])
                
                # Ako je detektovan objekat klase '15' (osoba) i konfidencija je veća od 0.5
                if idx == 15 and confidence > 0.5:
                    # Izračunaj koordinate pravougaonika oko detektovane osobe
                    box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
                    (startX, startY, endX, endY) = box.astype("int")
                    
                    # Nacrtaj pravougaonik oko detektovane osobe
                    cv.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
                    
                    # Označi da je osoba detektovana
                    person_detected = True

            # Dodaj rezultat detekcije (True/False) u međuspremnik
            self.buffer.append(person_detected)

            # Provjeri da li je osoba detektovana u bilo kojem od nedavnih kadrova
            if sum(self.buffer) > 0:  
                # Ako nije postavljen izlaz za video, postavi ga
                if self.out is None:
                    # Dobavi trenutni datum i vrijeme i formatiraj ih za ime fajla
                    now = datetime.datetime.now()
                    formatted_now = now.strftime("%d-%m-%y-%H-%M-%S")
                    print("Person motion detected at", formatted_now)
                    
                    # Kreiraj naziv za snimljeni video fajl
                    current_recording_name = os.path.join('detected_videos', f'{formatted_now}.mp4')
                    fourcc = cv.VideoWriter_fourcc(*'mp4v')
                    # Inicijalizuj video zapisnik sa zadatom brzinom kadrova i veličinom kadra
                    self.out = cv.VideoWriter(current_recording_name, fourcc, self.frame_rate, (frame.shape[1], frame.shape[0]))
                # Zapiši trenutni kadar u video fajl
                self.out.write(frame)
            else:
                # Ako više nema detekcije, oslobodi resurse video zapisnika
                if self.out is not None:
                    self.out.release()
                    self.out = None
                    print("Recording stopped")
            
            # Prikazivanje trenutnog kadra u prozoru
            cv.imshow('Camera Feed', frame)

            # Ako je pritisnuto dugme 'q', prekini petlju
            if cv.waitKey(1) & 0xFF == ord('q'):
                break

        # Oslobodi resurse video zapisnika ako su još uvek zauzeti
        if self.out is not None:
            self.out.release()
            self.out = None
            
        # Oslobodi resurse kamere i zatvori sve OpenCV prozore
        self.cap.release()
        cv.destroyAllWindows()
        print("Camera released...")



def main():
    # Kreiraj instancu klase Camera i pokreni metodu run
    camera = Camera()
    camera.run()

if __name__ == "__main__":
    main()

